# Measuring LLM Performance

In this notebook, we use the bespoke tooling to perform experiments to understand how to build a METCLOUD chatbot.
Firstly, we define a METCLOUD test set; this consists of 119 questions and answer pairs that we use to test Language Models deployed locally on the machine. We then set up a series of experiments to:
1. Ask a given LLM the questions via 'ollama'
2. Collect the responses
3. Compare/judge the comparison between the LLM response and the ground-truth

These experiments are varied as follows:
1. We do this for different open-source language models: Phi-3, Llama 3, Llama 3.1, Qwen-2, Mistral 7B and Gemma 2
2. We include no RAG, Naive RAG and Advanced RAG for each language model
3. For Naive RAG and Advanced RAG, we use different retrieval datasets AND different embedding models
4. We repeat the experiments using six _fine-tuned_ versions of the above models

This allows us to collect comprehensive performance metrics for each setup to determine the best core Chatbot pipeline!


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import ollama
from pathlib import Path

from src.qa import QuestionAnswering
from src.vectordb import ChromaDB
from src.verifier import Verifier
from src.reporting import folder_to_dataframe #collate results

/usr/local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Custom Tools

### `QuestionAnswering`
This class takes a pandas dataframe containing our question/answer pairs in our 119 test set and decomposes them into records. It will then loop through each record, and use `ollama-python` to ask a LLM the question, before collecting its response and storing it in a new dataframe alongside the original question and answer.
- System prompt is as follows:
  ```You are to be a human-like, compassionate, friendly and polite
                You are to be a human-like, compassionate, friendly and polite
                chatbot assistant for a cyber security firm. You will be asked customer support
                questions and it is your job to answer those questions. You aim to answer all
                queries, and if you are unsure you will ask the customer to hold while they
                are transferred to a human agent.
- The user prompt is the question!
- This tool can be passed a `ChromaDB` object to enable RAG.
- By default, this will add the answer from the closest matching question as context to the user prompt
- Advanced adopts re-ranking and the top three question answers

### `ChromaDB`
ChromaDB is a class to wraps around a `Chroma` object. This can be passed a DataFrame and the name of a `HuggingFace` embedding function, and it will vector embed our Q&A pairs in the dataframes as documents, where the vector is the `question` and the answer is stored as metadata.
- Implements a `retrieve` function which will take a prompt and return the metadata of the closest matching prompt using the embedding model
- This will add the top 1 matching questions' answer to the `user-prompt` for NAIVE mode
- Implements a re-ranker approach if requested using a CrossEncoder i.e adding top 3 _reranked_ questions answers to the user prompt for ADVANCED

### `Verifier`
The verifier is another wrapper around Ollama. Its sole purpose is to take two pieces of information and compare them for similarity and agreement. This uses chain-of-thought prompting and justification-forcing to improve the performance. We use this to take each answer and generated-answer and compare them for consistency! This uses an underlying language model to do this;
- System prompt is now much more complicated; CoT comes from the request for justification; a novel way to do CoT!
  ```# YOUR ROLE
    You are a question and answering validation capability. You can accurately compare two potential pieces of text for similarity and consistency.
    
    # YOUR TASK
    Your task is to assess / judge if information A : <gen_response>
    IS CONSISTENT with information B: <ground_truth>
    Information B should be treated as the TRUTH even if you disagree with its content. If the text samples contain similar and non-conflicting information, then then you should judge them as consistent. 
    
    # OUTPUT INSTRUCTIONS
    Return your judgement as a JSON compatible dictionary. An example of this format is:
    
    {"consistent": "(either \"True\" or \"False\")", "justification": "(description why the samples are consistent)"}
    
    Your output should only contain the "consistent" and "justification". Do not act as an assistant and do not yap. Make sure your output is valid JSON.```
- User prompt is as simple as `please compare this information`!
- We use regex to guarantee that the LLM has returned information in the format we expect.

We do this twice using different LLM's and take the average accuracy to test its efficacy


# Running the Experiments

We have six off-the-shelf LLMs and six finetuned LLMs (trained using the full 2500 metcloud dataset (via Unsloth))

We want to understand:
- The raw performance (i.e No RAG)
- Performance with Naive RAG and Advanced RAG, comparing:
  - Two different embedding functions for RAG (MiniLM, mpnet)
  - Using test dataset as lookup; using remainder dataset as lookup

Therefore, for each LLM, we have NINE sets of results. (i.e 8 RAG, one Raw)
We use the code below to define an experiment
   


In [3]:
#this is the path i.e. location to where our datasets are stored on our PC
data_path = Path('/data/')
core_dir  = data_path/'Demonstration' #folder where we save everything during experiment

#now we set our 'experiment' parameters
rag       = True                            #this means that we do use RAG
mode      = 'test'                          #this means we use the test set i.e. 119 questions for rag
advanced  = False                           #this means we use RERANKING when doing RAG i.e ADVANCED otherwise NAIVE.
emb_model = "all-MiniLM-L6-v2"              #this is the RAG embedding model
#emb_model = "all-mpnet-base-v2"

if rag == True:
    #load data to store in vector database for RAG - context
    if mode == "test":
        rag_data = pd.read_csv(data_path/'metcloud-with-id.csv')  #put test set into vector database (119)
    else:
        rag_data = pd.read_csv(data_path/'METCLOUD_training.csv') #removed the 119 questions in large dataset (2500 - 119) for rag never used full 2500

    #we now create a vector database of Q's using our ChromaDB class that we wrote
    #firstly, we create a cache folder to store our embedding model
    #then we pass our dataset and chroma will automatically embed the questions
    chroma_cache = data_path/f'chroma_cache/chromadb_{mode}_{emb_model}'
    chroma_db    = ChromaDB(chroma_cache,
                            rag_data,
                            embedding_model = emb_model)
    
    #create directories to store our generated answers
    save_dir = core_dir/f'dataset_{mode}_emb_{emb_model}_rerank_{advanced}'
    save_dir.mkdir(exist_ok=True,parents=True) #make_directory
else:
    #we do not use RAG!
    chroma_db = None
    save_dir = core_dir/'no_rag'
    save_dir.mkdir(exist_ok=True,parents=True) #make_directory

#119 questions to run through pipeline (always test set)
data_df =  pd.read_csv(data_path/'metcloud-with-id.csv')

#print out information
print('mode:',mode)
print('embedding_model:',emb_model)
print('rag:',rag)
print('advanced:',advanced)
print('chromadb:',chroma_db)
print('save_dir:',save_dir)

#three questions for demonstration
data_df = data_df.head(3)
print(data_df.shape)

/usr/local/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


mode: test
embedding_model: all-MiniLM-L6-v2
rag: True
advanced: False
chromadb: <src.vectordb.ChromaDB object at 0x7f090d286750>
save_dir: /data/Demonstration/dataset_test_emb_all-MiniLM-L6-v2_rerank_False
(3, 4)


In [4]:
data_df

,question,context,response,id
0,What is METCLOUD?,METCLOUD is a multi-award-winning secure sover...,METCLOUD is a secure sovereign cloud service p...,69a9382c7a9840248efc5d8851750530
1,How to contact METCLOUD?,If you have a question about how to easily ado...,"To contact METCLOUD, you can reach out to them...",327820806ca0495ca8360551897782c6
2,What are essential reasons to choose METCLOUD ...,METCLOUD 'Get Connected Cyber Safe' is our tra...,Choosing METCLOUD powered by HPE GreenLake for...,6d1765b7c3cc430196699c2b4e6c7e19


Quickly check that the chromadb is working as expected. Ask a question, see if we get an appropriate qa pair back

In [6]:
chroma_db.retrieve('what is metcloud',k=1)

["METCLOUD is a secure sovereign cloud service provider that specializes in offering digital modernization through advanced cybersecurity and artificial intelligence. It is designed to support businesses in adopting next-generation technologies for cloud computing and cybersecurity, ensuring that they stay secure, effective, and efficient. METCLOUD's approach is tailored to meet the unique needs of businesses, with a focus on a people-first strategy. The platform is scalable, making it suitable for small to medium-sized enterprises, and it has been recognized for its excellence in the field, including being named the Cybersecurity Firm of the Year by Finance Monthly in the 2021 FinTech Awards."]

### Now we run the Question and Answering loop!
In this cell, we do the following:
1. Define a list of open source models, available on Ollama.
2. Write a for loop to go through each model.
3. 'pull' the model -> this downloads it, if we don't already have it
4. Creates a `QuestionAnswering` class.
5. Processes the data
6. Asks each question in the dataset and stores the results to our 'save_dir' set earlier. We can pass in our Chromadb to enable RAG. This will be either niave or advanced, depending on the 'advanced' flag set earlier. 

In [7]:
ollama.list()

{'models': [{'name': 'metcloud_1epoch_Qwen2-7B-instruct-bnb-4bit:latest',
   'model': 'metcloud_1epoch_Qwen2-7B-instruct-bnb-4bit:latest',
   'modified_at': '2024-09-01T20:32:24.9618355Z',
   'size': 4683072814,
   'digest': '68c62ba5086d21af76fbd5687a392602e7231f9bc77190f9ad3ba442c99ab697',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'qwen2',
    'families': ['qwen2'],
    'parameter_size': '7.6B',
    'quantization_level': 'Q4_K_M'}},
  {'name': 'metcloud_1epoch_Phi-3-mini-4k-instruct-bnb-4bit:latest',
   'model': 'metcloud_1epoch_Phi-3-mini-4k-instruct-bnb-4bit:latest',
   'modified_at': '2024-09-01T20:31:35.465515Z',
   'size': 2318921171,
   'digest': '05154fcab5df76c54623459893db627ac9addf51c8cb538177dbfc49429fc870',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '3.8B',
    'quantization_level': 'Q4_K_M'}},
  {'name': 'metcloud_1epoch_mistral-7b-instruct-v0.3-bnb-4bit:lat

In [8]:
#off the shelf models
#models = ['phi3','mistral','gemma2','llama3','llama3.1','qwen2']

#demonstration, commented out other models

#finetuned models
#models = [i['name'] for i in ollama.list()['models'] if 'metcloud' in i['name']]

models = ['llama3.1']

#loop through each of the models in models
for model in models:
  #download the model if we dont have it  
  #ollama.pull(model)
  print('MODEL:',model)

  #create question/answering class, sourced at top of notebook
  qa = QuestionAnswering(model=model)

  #process the dataset into a list
  qa.process_dataset(data_df)

  #ask all questions, saving responses to a .csv file in save_dir
  qa.ask_all_questions(save_dir,
                       vector_db=chroma_db,
                       advanced = advanced)

MODEL: llama3.1


100%|██████████| 3/3 [02:57<00:00, 59.13s/it] 


### Now we do verification i.e. how good were the LLM responses?

1. We grab our answers generated in the previous cell
2. We define a set of 'verification' models. In this case, llama3.1 and gemma2
3. Loop through these models, create a verifier
4. Loop through the generated answers and check if the generated answers were good using verifier
5. Save results

In [9]:
#this runs after top half has completed all variations of experiments
#We first pick which set to verify
#advanced = True #False (naive)
#mode   = 'test'
#emb_model = "all-MiniLM-L6-v2" #fastest
#data_path = Path('/data/')

#this is the list of tested models
#models = ['gemma2','phi3','qwen2','llama3','llama3.1','mistral']

#now we define our verifier models i.e vmodels
vmodels = ['llama3.1','gemma2']

#we loop through our verifier models. This is just two loops
for vmodel in vmodels:
  #makes sure we definitely have the verifier model  
  #ollama.pull(vmodel)

  #now we create a verifier class, using our verifier model
  verifier = Verifier(vmodel)

  #'this' is just a directory where we will store our results
  #this  = f'dataset_{mode}_emb_{emb_model}_rerank_{advanced}'
  this  = data_path/f'Verification/{this}'
  this.mkdir(exist_ok=True,parents=True)

  #loop through model folders in the question answering directory 
  for model in models:

    #create a directory in our verifier directory to store these results
    pth = this/f'{vmodel}'
    pth.mkdir(exist_ok=True,parents=True)

    #load the csv containing all of the results
    answer_df = pd.read_csv(f'{save_dir}/{model}/all_questions.csv')

    #get verifier to see how good the responses were
    verifier.judge_all_questions(answer_df,model,pth)

100%|██████████| 3/3 [00:42<00:00, 14.15s/it]


### Reporting the Performance

Now we have built a method that looks into our verification folder and pulls out all of the performance metrics for each verifier run per model. We get the time, tokens per second and the accuracy averaged across the two models!

In [10]:
folder_to_dataframe(this,model_list=models)

,model,time,tps,gemma2_accuracy,llama3.1_accuracy,average_accuracy
0,llama3.1,59.099569,25.984397,1.0,1.0,1.0


In [15]:
folder_to_dataframe('/data/Verification/no_rag/',model_list=['phi3','llama3.1','gemma2','qwen2','llama3','mistral'])

,model,time,tps,gemma2_accuracy,llama3.1_accuracy,average_accuracy
0,gemma2,7.174388,22.264301,0.747899,0.672269,0.710084
1,llama3.1,6.479725,32.016442,0.890756,0.957983,0.924370
2,llama3,7.445677,31.509900,0.840336,0.823529,0.831933
3,mistral,6.168018,33.226657,0.857143,0.873950,0.865546
4,phi3,7.622237,51.759784,0.848739,0.949580,0.899160
5,qwen2,7.666361,33.297824,0.907563,0.974790,0.941176


In [3]:
benchmarking_folder = Path('/data/Benchmarking/')
results_folder      = Path('/data/Results/')

vmodels = ['gemma2','llama3.1']

for exp in benchmarking_folder.iterdir():
    if '.ipynb' in exp.name: continue
    exp_name = exp.stem

    for vmodel in vmodels:
        verifier = Verifier(model = vmodel)
        save_folder = results_folder/exp_name
        save_folder.mkdir(exist_ok=True,parents=True)

        for model in exp.iterdir():
            if '.ipynb' in model.name: continue
            model_name = model.name
   
            question_df = pd.read_csv(model / 'all_questions.csv')

            save_dir = save_folder / vmodel
            save_dir.mkdir(exist_ok=True,parents=True)
            if (save_dir/f'{model_name}.csv').is_file():
                print('Skipping!')
                continue

            verifier.judge_all_questions(question_df, model_name, save_dir)

/usr/local/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Skipping!
Skipping!
Skipping!
Skipping!
Skipping!
Skipping!
Skipping!
Skipping!
Skipping!
Skipping!
Skipping!
Skipping!


/usr/local/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Skipping!
Skipping!
Skipping!
Skipping!
Skipping!
Skipping!
Skipping!
Skipping!
Skipping!
Skipping!
Skipping!
Skipping!


/usr/local/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

/usr/local/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Skipping!
Skipping!
Skipping!
Skipping!
Skipping!
Skipping!
Skipping!
Skipping!
Skipping!
Skipping!
Skipping!
Skipping!


/usr/local/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Skipping!
Skipping!
Skipping!
Skipping!
Skipping!
Skipping!
Skipping!


100%|██████████| 119/119 [04:42<00:00,  2.37s/it]
/usr/local/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
100%|██████████| 119/119 [08:36<00:00,  4.34s/it]
/usr/local/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
100%|██████████| 119/119 [04:43<00:00,  2.39s/it]
/usr/local/lib/python3.12/site-packages/transformers/tokenization_util